In [1]:
import os
import chromadb
from typing_extensions import Annotated
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

config_list = [
    # {
    #     # Let's choose the Llama 3 model
    #     "model": "llama3-8b-8192",
    #     # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
    #     "api_key": os.environ["GROQ_API_KEY"],
    #     # We specify the API Type as 'groq' so it uses the Groq client class
    #     "api_type": "groq",
    # },
    {
       # Let's choose the Llama 3 model
        "model": "llama3-70b-8192",
        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": os.environ["GROQ_API_KEY"],
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq", 
    }
]

llm_config = {
    "config_list":config_list,
    "temperature": 0.1,
    "seed": 1234,
}

In [2]:
print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

LLM models:  ['llama3-70b-8192']


Constructing agents

In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

designer = UserProxyAgent(
    name="Designer",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config= {
        "work_dir": "tmp/NewCADs",
        "use_docker": False,
    },
    llm_config={"config_list": config_list},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    description= "The designer who asks questions to create CAD models using CadQuery",
    default_auto_reply="Reply `TERMINATE` if the task is done.",
)

designer_aid  = RetrieveUserProxyAgent(
    name="Designer_Assistant",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    llm_config={"config_list": config_list},
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    code_execution_config=False,
    retrieve_config={
        "task": "code",
        "docs_path":[
            "/home/niel77/MechanicalAgents/data/Examples_small.md",
            ],
        "chunk_token_size" : 500,
        "collection_name" : "groupchat",
        "get_or_create": True,
    },
)

cad_coder = AssistantAgent(
    "CadQuery Code Writer",
    system_message= '''You are a CadQuery expert specializing in creating CAD models using Python. Follow the exact structure and format provided below to solve design problems and save the CAD models in STL, STEP, and DXF formats. Adhere strictly to the following outline for every response:

1. **Import Libraries:**
   Always include necessary imports, especially `cadquery` and `ocp_vscode` for model visualization.

2. **Define Parameters:**
   Clearly define parameters for the model, such as dimensions and other properties.

3. **Create the CAD Model:**
   Use CadQuery functions to build the CAD model based on the defined parameters.

4. **Save the Model:**
   Save the model in STL, STEP, and DXF formats using `cq.exporters.export`.

5. **Visualize the Model:**
   Use `show()` from the `ocp_vscode` library to visualize the created model.

6. **Example Structure:**
   ```python
   import cadquery as cq
   from ocp_vscode import *  # Always include this for visualization.

   # Step 1: Define Parameters
   height = 60.0
   width = 80.0
   thickness = 10.0

   # Step 2: Create the CAD Model
   box = cq.Workplane("XY").box(height, width, thickness)

   # Step 3: Save the Model
   cq.exporters.export(box, "box.stl")
   cq.exporters.export(box.section(), "box.dxf")
   cq.exporters.export(box, "box.step")

   # Step 4: Visualize the Model
   show(box)  # Always use this to visualize the model.
''',
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    description="CadQuery Code Writer who writes python code to create CAD models following the system message.",
)

reviewer = AssistantAgent(
    name="Code Reviewer",
    is_termination_msg=termination_msg,
    system_message=''' You are a code review expert. Your role is to ensure that the "Creator" agent's response follows the exact structure and format specified. Check the response against the following guidelines:
    Import Libraries: Verify that the necessary imports, including cadquery and ocp_vscode, are included.
    Define Parameters: Ensure all necessary parameters (e.g., dimensions) are defined clearly.
    Create the CAD Model: Confirm that the model is built correctly using the provided parameters.
    Save the Model: Make sure the model is saved in STL, STEP, and DXF formats.
    Visualize the Model: Check if show() is used for visualization.
    If any step is missing, incorrect, or not following the instructions, provide constructive feedback to the "Creator" agent to correct it.
    Reply `TERMINATE` in the end when everything is done. ''' ,
    llm_config=llm_config,
    description="Code Reviewer who can review the python code created to generate CAD models using CadQuery and also visualize using show(model) method.",
    code_execution_config= {
        "work_dir": "tmp/NewCADs",
        "use_docker": False,
    },
)

/home/niel77/miniconda3/envs/magents/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def _reset_agents():
    designer.reset()
    designer_aid.reset()
    cad_coder.reset()
    reviewer.reset()

In [5]:
def rag_chat(design_problem : str):
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[designer_aid,  cad_coder, reviewer], messages=[], max_round=12, speaker_selection_method="round_robin"
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    designer_aid.initiate_chat(
        manager,
        message=designer_aid.message_generator,
        problem=design_problem,
        n_results=3,
    )

In [6]:
def norag_chat(design_prblem: str):
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[designer, cad_coder, reviewer,],
        messages=[],
        max_round=12,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=False,
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with the boss as this is the user proxy agent.
    designer.initiate_chat(
        manager,
        message=design_prblem,
    )

In [7]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={"config_list": config_list},
)

from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent



In [8]:
def call_rag_chat(design_problem : str):
    _reset_agents()

    # In this case, we will have multiple user proxy agents and we don't initiate the chat
    # with RAG user proxy agent.
    # In order to use RAG user proxy agent, we need to wrap RAG agents in a function and call
    # it from other agents.
    def retrieve_content(
        message: Annotated[
            str,
            "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
        ],
        n_results: Annotated[int, "number of results"] = 3,
    ) -> str:
        designer_aid.n_results = n_results  # Set the number of results to be retrieved.
        _context = {"problem": message, "n_results": n_results}
        ret_msg = designer_aid.message_generator(designer_aid, None, _context)
        return ret_msg or message

    designer_aid.human_input_mode = "NEVER"  # Disable human input for boss_aid since it only retrieves content.

    for caller in [cad_coder, reviewer]:
        d_retrieve_content = caller.register_for_llm(
            description="retrieve content for code generation and CAD model generation", api_style="function"
        )(retrieve_content)

    for executor in [designer]:
        executor.register_for_execution()(d_retrieve_content)

    groupchat = autogen.GroupChat(
        agents=[designer, cad_coder, reviewer],
        messages=[],
        max_round=12,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=False,
    )

    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with the boss as this is the user proxy agent.
    designer.initiate_chat(
        manager,
        message=design_problem,
    )

In [9]:
design_problem ="Create a plate of dimension 100*200 with thickness 5mm having a central hole of diameter 20mm."

In [12]:
norag_chat(design_problem)

Designer (to chat_manager):

Create a plate of dimension 100*200 with thickness 5mm having a central hole of diameter 20mm.

--------------------------------------------------------------------------------

Next speaker: CadQuery Code Writer

CadQuery Code Writer (to chat_manager):

Here is the CadQuery code to create a plate with a central hole:
```python
import cadquery as cq
from ocp_vscode import *

# Step 1: Define Parameters
length = 200.0
width = 100.0
thickness = 5.0
hole_diameter = 20.0

# Step 2: Create the CAD Model
plate = cq.Workplane("XY").box(length, width, thickness)
hole = plate.cut(cq.Workplane("XY").circle(hole_diameter/2).extrude(thickness))

# Step 3: Save the Model
cq.exporters.export(hole, "plate.stl")
cq.exporters.export(hole.section(), "plate.dxf")
cq.exporters.export(hole, "plate.step")

# Step 4: Visualize the Model
display(hole) # for jupyter lab
show(hole)  # Always use this to visualize the model.
```
This code creates a rectangular plate with a central ho

In [10]:
rag_chat("Create a plate of dimension 100*200 with thickness 5mm having a central hole of diameter 20mm.")

Trying to create collection.


2024-10-02 17:33:39,914 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 4 chunks.
2024-10-02 17:33:39,918 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


VectorDB returns doc_ids:  [['4dec1a57', 'db147485', '23851451']]
Adding content of doc 4dec1a57 to context.
Adding content of doc db147485 to context.
Adding content of doc 23851451 to context.
Designer_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Create a plate of dimension 100*200 with thickness 5mm having a central hole of diameter 20mm.

Context is: [(1.5, 0), (0, 1.5), (-1.5, 0), (0, -1.5)]

) # now four points are on the stack

r = r.circle(0.25) # circle will operate on all four points

In [9]:
from autogen import ConversableAgent
Cad_codewriter = ConversableAgent(
    "CAD Code Writer",
    system_message='''CAD Code Writer.You are a CadQuery expert and you write codes in Python to create CAD models using CADquery. 
        Here is an example of abox you created and saved in the step, dxf and stl format.
        ##
        Q: Create a box of size 80*60*10 and save it in stl, step and dxf file format.
        A:
        ```python
        import cadquery as cq
        from ocp_vscode import * # this is used to visualize model with OCP CAD viewer
        height = 60.0
        width = 80.0
        thickness = 10.0
        # make the base
        box = cq.Workplane("XY").box(height, width, thickness)
        show(box)
        cq.exporters.export(box, "box.stl")
        cq.exporters.export(box.section(), "box.dxf")
        cq.exporters.export(box, "box.step")
        ##
        ```
        '''
        ,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
)

In [10]:
import autogen
group_chat = autogen.GroupChat(agents=[user_proxy_agent, Cad_codewriter, ragproxyagent],speaker_selection_method= 'round_robin', messages=[], max_round=4)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list})



In [ ]:
user_proxy_agent.initiate_chat(
    manager,
    message="Write the CAdQuery Code to create a plate with hole.",
    silent = False,
    max_turns=10,
)

In [19]:
import cadquery as cq
from ocp_vscode import * # this is used to visualize model with OCP CAD viewer
height = 60.0
width = 80.0
thickness = 10.0
# make the base
box = cq.Workplane("XY").box(height, width, thickness)
show(box)

Using port 3939 taken from config file
Jupyter kernel running
Jupyter connection file path written to /home/niel77/.ocpvscode
+

In [27]:
box

In [28]:
display("/home/niel77/MechanicalAgents/tests/mesh.tjs")

'/home/niel77/MechanicalAgents/tests/mesh.tjs'

In [26]:
from cadquery import exporters
box.export(
    "mesh.json",
    tolerance=0.01,
    angularTolerance=0.1,
    exportType=exporters.ExportTypes.TJS,
)

TypeError: Workplane.export() got an unexpected keyword argument 'exportType'